In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [ ]:
#load in the Data
df_sample = pd.read_csv('/kaggle/input/electricity-shortfall-challenge/sample_submission_load_shortfall.csv')
df_train = pd.read_csv('/kaggle/input/electricity-shortfall-challenge/df_train.csv')
df_test = pd.read_csv('/kaggle/input/electricity-shortfall-challenge/df_test.csv')

In [ ]:
df_sample.head(3)

In [ ]:
df_train.head(3)

In [ ]:
df_test.head(3)

In [ ]:
#see how many columns and what they are
df_train.columns

In [ ]:
#see some basic understanding of the dataset
df_train.describe()

In [ ]:
#find all the nulls
df_train.isnull().sum()

In [ ]:
#merge the train and test datasets
#simplify work when editing the data
#data sits on top of each other - train at top and test at bottom
df = pd.concat([df_train, df_test])
df.head(3)

In [ ]:
df.dtypes

In [ ]:
#see the last part of the data aka the test set
#last column is NaN as that is the column we are going to forcast
df.tail(5)

In [ ]:
df.isnull().sum()

In [ ]:
#Valencia Pressure has the most NaN (excluding last column which we trying to forcast)
sns.boxplot(df['Valencia_pressure'])

In [ ]:
print('Mode')
print(df['Valencia_pressure'].mode())
print('Mean')
print(df['Valencia_pressure'].mean())
print('Median')
print(df['Valencia_pressure'].median())

In [ ]:
#do not lose original dataset, create a new dataset 
df_clean = df
#replace all the null values with the mode for valencia_pressure 
df_clean['Valencia_pressure'] = df_clean['Valencia_pressure'].fillna(df_clean['Valencia_pressure'].mode()[0])

In [ ]:
#see if all the null values have been filled
df_clean.isnull().sum()

In [ ]:
df_clean['Valencia_wind_deg'] = df_clean['Valencia_wind_deg'].str.extract('(\d+)')

In [ ]:
df_clean['Valencia_wind_deg'] = pd.to_numeric(df_clean['Valencia_wind_deg'])

In [ ]:
df_clean['Seville_pressure'] = df_clean['Seville_pressure'].str.extract('(\d+)')

In [ ]:
df_clean['Seville_pressure'] = pd.to_numeric(df_clean['Seville_pressure'])

In [ ]:
df_clean.dtypes

In [ ]:
df_clean.head(3)

In [ ]:
df_clean['Valencia_wind_deg']

In [ ]:
df_clean['time'] = pd.to_datetime(df_clean['time'])

In [ ]:
df_clean = df_clean.drop(['Unnamed: 0', 'time'], axis =1)

In [ ]:
df_clean['Year'] = pd.DatetimeIndex(df['time']).year

In [ ]:
df_clean['Month'] = pd.DatetimeIndex(df['time']).month

In [ ]:
df_clean['Day'] = pd.DatetimeIndex(df['time']).day

In [ ]:
df_clean.dtypes

In [ ]:
df_clean.columns

In [ ]:
df_clean.head()

In [ ]:
df_clean.corr()

In [ ]:
y = df_clean[:len(df_train)][['load_shortfall_3h']]
x = df_clean[:len(df_train)].drop('load_shortfall_3h', axis = 1)

In [ ]:
x.head()

In [ ]:
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import LogisticRegression


In [ ]:
Lr = LinearRegression()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.20)

In [ ]:
Lr.fit(x_train,y_train)
preds = Lr.predict(x_test)


In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
def rmse(y_test, y_predict):
    return np.sqrt(mean_squared_error(y_test, y_predict))

In [ ]:
rmse(y_test, preds)

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
r2_score(y_test, preds)

In [ ]:
x_train = df_clean[:len(df_train)].drop('load_shortfall_3h', axis =1)
x_test = df_clean[:len(df_train)].drop('load_shortfall_3h', axis =1)

In [ ]:
Lr.fit(x_train, y)
preds = Lr.predict(x_test)

In [ ]:
daf=pd.DataFrame(preds, columns =['load_shortfall_3h'])
daf.head()

In [ ]:
output = pd.DataFrame({"time":df_test['time']})
submission = output.join (daf)
submission.to_csv("submission.csv", index=False)

In [ ]:
submission